Upang mapatakbo ang mga sumusunod na notebook, kung hindi mo pa nagagawa, kailangan mong itakda ang openai key sa loob ng .env file bilang `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Susunod, ilalagay natin ang Embedding Index sa isang Pandas Dataframe. Ang Embedding Index ay naka-imbak sa isang JSON file na tinatawag na `embedding_index_3m.json`. Ang Embedding Index ay naglalaman ng mga Embeddings para sa bawat YouTube transcript hanggang huling bahagi ng Oktubre 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Susunod, gagawa tayo ng function na tinatawag na `get_videos` na maghahanap sa Embedding Index gamit ang query. Ibabalik ng function ang top 5 na videos na pinaka-katulad sa query. Ganito ang proseso ng function:

1. Una, gagawa ng kopya ng Embedding Index.
2. Sunod, kakalkulahin ang Embedding para sa query gamit ang OpenAI Embedding API.
3. Pagkatapos, maglalagay ng bagong column sa Embedding Index na tinatawag na `similarity`. Ang column na `similarity` ay naglalaman ng cosine similarity sa pagitan ng query Embedding at ng Embedding ng bawat video segment.
4. Sunod, ifi-filter ang Embedding Index gamit ang column na `similarity`. Ipi-filter ang Embedding Index para isama lang ang mga video na may cosine similarity na 0.75 o mas mataas pa.
5. Sa huli, ise-sort ang Embedding Index ayon sa column na `similarity` at ibabalik ang top 5 na videos.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ang function na ito ay napakasimple, ipinapakita lamang nito ang mga resulta ng query sa paghahanap.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Una, ang Embedding Index ay niloload sa isang Pandas Dataframe.
2. Sunod, hihingan ang user na maglagay ng query.
3. Pagkatapos, tatawagin ang function na `get_videos` para hanapin sa Embedding Index ang query.
4. Sa huli, tatawagin ang function na `display_results` para ipakita ang mga resulta sa user.
5. Hihingan ulit ang user na maglagay ng panibagong query. Paulit-ulit ang prosesong ito hanggang sa mag-type ang user ng `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.tl.png)

Hihingan ka na maglagay ng query. Maglagay ng query at pindutin ang enter. Magbabalik ang application ng listahan ng mga video na may kaugnayan sa query mo. Magbibigay din ang application ng link papunta sa mismong bahagi ng video kung saan matatagpuan ang sagot sa tanong.

Narito ang ilang mga query na pwede mong subukan:

- Ano ang Azure Machine Learning?
- Paano gumagana ang convolutional neural networks?
- Ano ang neural network?
- Pwede ba akong gumamit ng Jupyter Notebooks sa Azure Machine Learning?
- Ano ang ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama’t nagsusumikap kami para sa kawastuhan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi eksaktong pagsasalin. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring lumitaw mula sa paggamit ng pagsasaling ito.
